# Comparativa de Parroquias entre Hojas

Este script compara las parroquias de la Hoja 1 con la Hoja 2 del archivo `data/comparativa.xlsx` y muestra a qué sector corresponde cada parroquia.

In [24]:
import pandas as pd

# Cargar el archivo Excel con ambas hojas
file_path = 'data/comparativo.xlsx'

# Leer ambas hojas
hoja1 = pd.read_excel(file_path, sheet_name=0)  # Primera hoja
hoja2 = pd.read_excel(file_path, sheet_name=1)  # Segunda hoja

print("Hoja 1:")
print(hoja1.head())
print(f"\nTotal de registros en Hoja 1: {len(hoja1)}")
print("\n" + "="*80 + "\n")
print("Hoja 2:")
print(hoja2.head())
print(f"\nTotal de registros en Hoja 2: {len(hoja2)}")

Hoja 1:
   SECTOR    Parroquia
0  CENTRO        QUITO
1  CENTRO    RUMIPAMBA
2   NORTE     CALDERON
3   NORTE  CHAVEZPAMBA
4   NORTE       GUALEA

Total de registros en Hoja 1: 40


Hoja 2:
           Parroquia  SECTOR
0           Alangasí  VALLE1
1           Amaguaña  VALLE1
2          Atahualpa   RURAL
3  Belisario Quevedo   NORTE
4           Calacalí   RURAL

Total de registros en Hoja 2: 68


In [25]:
# Realizar la comparativa correcta considerando duplicados
# Hacer merge por Parroquia Y Sector para encontrar coincidencias exactas
comparativa = hoja1.merge(
    hoja2, 
    left_on=['Parroquia', 'SECTOR'], 
    right_on=['Parroquia', 'SECTOR'],
    how='left',
    indicator=True,
    suffixes=('_Hoja1', '_Hoja2')
)

# Renombrar columnas para claridad
comparativa['Estado'] = comparativa['_merge'].map({
    'both': 'Coincide',
    'left_only': 'Diferente o No existe'
})

# Buscar el sector en Hoja 2 para cada parroquia (sin importar si coincide)
sector_dict_hoja2 = hoja2.groupby('Parroquia')['SECTOR'].apply(list).to_dict()
comparativa['Sectores_en_Hoja2'] = comparativa['Parroquia'].map(
    lambda x: ', '.join(sector_dict_hoja2.get(x, ['NO ENCONTRADA']))
)

print("Comparativa de Sectores:")
print(comparativa[['SECTOR', 'Parroquia', 'Sectores_en_Hoja2', 'Estado']])
print(f"\nTotal de registros comparados: {len(comparativa)}")

Comparativa de Sectores:
                       SECTOR                  Parroquia  \
0                      CENTRO                      QUITO   
1                      CENTRO                  RUMIPAMBA   
2                       NORTE                   CALDERON   
3                       NORTE                CHAVEZPAMBA   
4                       NORTE                     GUALEA   
5                       NORTE                LLANO CHICO   
6                       NORTE                    NANEGAL   
7                       NORTE                 NANEGALITO   
8                       NORTE                       NONO   
9                       NORTE                      PACTO   
10                      NORTE                    PERUCHO   
11                      NORTE                   POMASQUI   
12                      NORTE                   PUELLARO   
13                      NORTE                SAN ANTONIO   
14                      NORTE          SAN JOSE DE MINAS   
15             

In [26]:
# Análisis de coincidencias y diferencias
print("="*80)
print("\nANÁLISIS DE RESULTADOS\n")
print("="*80)

# Registros que coinciden exactamente (misma parroquia y mismo sector)
coinciden = comparativa[comparativa['Estado'] == 'Coincide']
print(f"\n✓ Registros con Parroquia Y Sector coincidentes: {len(coinciden)}")
if len(coinciden) > 0:
    print("\nDetalle:")
    print(coinciden[['SECTOR', 'Parroquia']])

# Registros diferentes o no encontrados
diferentes = comparativa[comparativa['Estado'] == 'Diferente o No existe']
print(f"\n⚠ Registros con diferencias: {len(diferentes)}")
if len(diferentes) > 0:
    print("\nDetalle:")
    print(diferentes[['SECTOR', 'Parroquia', 'Sectores_en_Hoja2']])


ANÁLISIS DE RESULTADOS


✓ Registros con Parroquia Y Sector coincidentes: 0

⚠ Registros con diferencias: 40

Detalle:
                       SECTOR                  Parroquia     Sectores_en_Hoja2
0                      CENTRO                      QUITO         NO ENCONTRADA
1                      CENTRO                  RUMIPAMBA         NO ENCONTRADA
2                       NORTE                   CALDERON         NO ENCONTRADA
3                       NORTE                CHAVEZPAMBA         NO ENCONTRADA
4                       NORTE                     GUALEA         NO ENCONTRADA
5                       NORTE                LLANO CHICO         NO ENCONTRADA
6                       NORTE                    NANEGAL         NO ENCONTRADA
7                       NORTE                 NANEGALITO         NO ENCONTRADA
8                       NORTE                       NONO         NO ENCONTRADA
9                       NORTE                      PACTO         NO ENCONTRADA
10         

In [27]:
# Análisis detallado de las diferencias
print("="*80)
print("\n🔍 ANÁLISIS DETALLADO DE DIFERENCIAS\n")
print("="*80)

# Separar los casos
solo_en_hoja1 = diferentes[diferentes['Sectores_en_Hoja2'] == 'NO ENCONTRADA']
sector_diferente = diferentes[diferentes['Sectores_en_Hoja2'] != 'NO ENCONTRADA']

print(f"\n1️⃣ Parroquias que NO existen en Hoja 2: {len(solo_en_hoja1)}")
if len(solo_en_hoja1) > 0:
    for idx, row in solo_en_hoja1.iterrows():
        print(f"   • {row['Parroquia']} (Sector en Hoja 1: {row['SECTOR']})")

print(f"\n2️⃣ Parroquias con SECTOR DIFERENTE entre hojas: {len(sector_diferente)}")
if len(sector_diferente) > 0:
    for idx, row in sector_diferente.iterrows():
        print(f"\n   🔸 {row['Parroquia']}")
        print(f"      Sector en Hoja 1: {row['SECTOR']}")
        print(f"      Sector(es) en Hoja 2: {row['Sectores_en_Hoja2']}")


🔍 ANÁLISIS DETALLADO DE DIFERENCIAS


1️⃣ Parroquias que NO existen en Hoja 2: 39
   • QUITO (Sector en Hoja 1: CENTRO)
   • RUMIPAMBA (Sector en Hoja 1: CENTRO)
   • CALDERON (Sector en Hoja 1: NORTE)
   • CHAVEZPAMBA (Sector en Hoja 1: NORTE)
   • GUALEA (Sector en Hoja 1: NORTE)
   • LLANO CHICO (Sector en Hoja 1: NORTE)
   • NANEGAL (Sector en Hoja 1: NORTE)
   • NANEGALITO (Sector en Hoja 1: NORTE)
   • NONO (Sector en Hoja 1: NORTE)
   • PACTO (Sector en Hoja 1: NORTE)
   • PERUCHO (Sector en Hoja 1: NORTE)
   • POMASQUI (Sector en Hoja 1: NORTE)
   • PUELLARO (Sector en Hoja 1: NORTE)
   • SAN ANTONIO (Sector en Hoja 1: NORTE)
   • SAN JOSE DE MINAS (Sector en Hoja 1: NORTE)
   • ZAMBIZA (Sector en Hoja 1: NORTE)
   • ATAHUALPA (Sector en Hoja 1: SUR)
   • LA MERCED (Sector en Hoja 1: SUR)
   • LLOA (Sector en Hoja 1: SUR)
   • TODO PINCHINCHA+RUMIÑAHUI (Sector en Hoja 1: TODO PINCHINCHA+RUMIÑAHUI)
   • ALANGASI (Sector en Hoja 1: VALLE)
   • AMAGUAÑA (Sector en Hoja 1: VALLE)


In [28]:
# Resumen de cambios de sectores
if len(sector_diferente) > 0:
    print("\n" + "="*80)
    print("\n📊 RESUMEN DE CAMBIOS DE SECTORES\n")
    print("="*80)
    
    print(f"\nTotal de registros con sectores diferentes: {len(sector_diferente)}\n")
    
    # Agrupar por cambio de sector
    cambios_resumen = {}
    for idx, row in sector_diferente.iterrows():
        key = f"{row['SECTOR']} → {row['Sectores_en_Hoja2']}"
        if key not in cambios_resumen:
            cambios_resumen[key] = []
        cambios_resumen[key].append(row['Parroquia'])
    
    for cambio, parroquias in sorted(cambios_resumen.items()):
        print(f"  • {cambio}")
        for p in parroquias:
            print(f"    - {p}")
        print()
else:
    print("\n✓ Todos los sectores coinciden entre las hojas.")



📊 RESUMEN DE CAMBIOS DE SECTORES


Total de registros con sectores diferentes: 1

  • VALLE → CENTRO NORTE, VALLE1
    - Rumipamba



In [29]:
# Exportar resultados a un archivo Excel
from datetime import datetime

# Generar nombre con timestamp para evitar conflictos
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f'data/resultado_comparativa_{timestamp}.xlsx'

try:
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        comparativa.to_excel(writer, sheet_name='Comparativa Completa', index=False)
        coinciden.to_excel(writer, sheet_name='Coinciden', index=False)
        diferentes.to_excel(writer, sheet_name='Diferentes', index=False)
        if len(sector_diferente) > 0:
            sector_diferente.to_excel(writer, sheet_name='Sectores Diferentes', index=False)
    
    print(f"\n✓ Resultados exportados a: {output_file}")
except PermissionError:
    print(f"\n⚠ Error: No se pudo guardar el archivo.")
    print("Posibles causas:")
    print("  - El archivo está abierto en Excel u otra aplicación")
    print("  - No tienes permisos de escritura en la carpeta")
    print(f"\nIntenta cerrar el archivo y volver a ejecutar esta celda.")


✓ Resultados exportados a: data/resultado_comparativa_20260116_155858.xlsx
